In [9]:
from elasticsearch import Elasticsearch, helpers
from tqdm import tqdm

client = Elasticsearch(
    hosts="http://localhost:9200"
)

client.info()

ObjectApiResponse({'name': '3f6292eb3d6f', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'bscp7RCYTm6aefi0n-kUpQ', 'version': {'number': '8.18.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '04e979aa50b657bebd4a0937389308de82c2bdad', 'build_date': '2025-04-10T10:09:16.444104780Z', 'build_snapshot': False, 'lucene_version': '9.12.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [10]:
# 將原本postgresql的data insert至elatic中
import psycopg2

endpoint = "localhost"
database = "postgres"
username = "postgres"
password = "mysecretpassword"
port = 8080
connection = psycopg2.connect(
    dbname=database,
    host=endpoint,
    port=port,
    user=username,
    password=password
)

cur = connection.cursor()
query = """SELECT id, text_segment
FROM public.section_content_charsplitt;"""
# query = """SELECT id, table_content
# FROM public.table_markdown_v2;"""
cur.execute(query)
result = cur.fetchall()

index_name = "chunk-index"
# index_name = "table-index"
docs = []
for i in result:
    docs.append({"id":i[0],"text":i[1]})

bulk_response = helpers.bulk(client, docs, index=index_name)
print(bulk_response)

(1921, [])


In [17]:
# Elasticsearch 的查詢 DSL
index_name = "chunk-index"
search_keyword = "教師介紹"
field_to_search = "text"
search_body = {
  "query": {
    "match": {
      field_to_search: search_keyword
    }
  },
  # 可選：添加其他參數，例如指定返回數量、分頁、高亮等
  "size": 3,       # 最多返回 10 筆結果
  "from": 0,      # 從第 0 筆結果開始 (用於分頁)
  "highlight": {  # 要求高亮顯示匹配的關鍵字
    "fields": {
      field_to_search: {}
    }
  }
}

response = client.search(index=index_name, body=search_body)
response.body

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 33, 'relation': 'eq'},
  'max_score': 4.9382,
  'hits': [{'_index': 'chunk-index',
    '_id': 'lDuOV5YBwXn6e0uC-oQ3',
    '_score': 4.9382,
    '_source': {'id': '20250403171853018627',
     'text': '。這兩場訓練不僅為參與者提供了深入的學習機會，更強化了 對 SAS 系統的瞭解與實際應用。同時，我們也撰寫了兩篇科普文章， 以清晰而易懂的方式介紹 SAS 系統的原理、應用範疇和未來趨勢，致\n力於推動相關領域的普及和瞭解。'},
    'highlight': {'text': ['同時，我們也撰寫了兩篇科普文章， 以清晰而易懂的方式<em>介紹</em> SAS 系統的原理、應用範疇和未來趨勢，致\n力於推動相關領域的普及和瞭解。']}},
   {'_index': 'chunk-index',
    '_id': 'zTuOV5YBwXn6e0uC-oQ3',
    '_score': 4.6952443,
    '_source': {'id': '20250403171938189691',
     'text': '生態環保標章(EU Ecolabel)推薦的國際安全替代資料庫清單， 以下為資料庫清單介紹：\n1. ChemSec Marketplace (https://marketplace.chemsec.org/)\n國際化學秘書處(The International Chemical Secretariat, ChemSec)於 2002 年在 ChemSec 建立的 ChemSec\nMarketplace 搜集來自不同企業、製造商、供應鏈對於產品中所使用化學品的技術 報告。ChemSec Marketplace 將這些技術報告分成三類： Requests、Alternative

In [18]:
type(response.body)
len(response.body['hits']['hits'])
response.body['hits']['hits']
for item in response.body['hits']['hits']:
    print(item['_source']['text'])
    print('========================================')
    

。這兩場訓練不僅為參與者提供了深入的學習機會，更強化了 對 SAS 系統的瞭解與實際應用。同時，我們也撰寫了兩篇科普文章， 以清晰而易懂的方式介紹 SAS 系統的原理、應用範疇和未來趨勢，致
力於推動相關領域的普及和瞭解。
生態環保標章(EU Ecolabel)推薦的國際安全替代資料庫清單， 以下為資料庫清單介紹：
1. ChemSec Marketplace (https://marketplace.chemsec.org/)
國際化學秘書處(The International Chemical Secretariat, ChemSec)於 2002 年在 ChemSec 建立的 ChemSec
Marketplace 搜集來自不同企業、製造商、供應鏈對於產品中所使用化學品的技術 報告。ChemSec Marketplace 將這些技術報告分成三類： Requests、Alternatives、 Evaluat
要 工具，在未來的化學領域中發揮越來越重要的作用。統整國際上常見的綠色替代 資料庫，參考歐盟生態環保標章(EU Ecolabel)推薦的國際安全替代資料庫清單， 以下為資料庫清單介紹：
1. ChemSec Marketplace (https://marketplace.chemsec.org/)
國際化學秘書處(The International Chemical Secretariat, ChemSec)於 2002 年在 ChemSec 建立的 ChemSec
Marketplace 搜集來自不同企業、製造商、供應鏈對於產品中所使用化學品的技術 報告。ChemSec Marketpla


In [19]:
# 查看分詞效果
text_to_analyze = "替代化學品有哪些物理性質需要考慮" 
# 使用 tokenizer
analyze_body_tokenizer = {
    "tokenizer": "ik_smart", 
    "text": text_to_analyze
}

response = client.indices.analyze(body=analyze_body_tokenizer)
response.body['tokens']

[{'token': '替代',
  'start_offset': 0,
  'end_offset': 2,
  'type': 'CN_WORD',
  'position': 0},
 {'token': '化學品',
  'start_offset': 2,
  'end_offset': 5,
  'type': 'CN_WORD',
  'position': 1},
 {'token': '有哪些',
  'start_offset': 5,
  'end_offset': 8,
  'type': 'CN_WORD',
  'position': 2},
 {'token': '物理性質',
  'start_offset': 8,
  'end_offset': 12,
  'type': 'CN_WORD',
  'position': 3},
 {'token': '需要',
  'start_offset': 12,
  'end_offset': 14,
  'type': 'CN_WORD',
  'position': 4},
 {'token': '考慮',
  'start_offset': 14,
  'end_offset': 16,
  'type': 'CN_WORD',
  'position': 5}]